Playing around with the `h5py` library...

In [4]:
import os

import h5py as h5
import numpy as np

from pathlib import Path as path
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap as bm

In [5]:
# create variables holding paths to input data
DATA_DIR = os.path.join('..', 'data')
INPUT_DATA_DIR = os.path.join(DATA_DIR, 'input')
SMAP_DATA_DIR = os.path.join(INPUT_DATA_DIR, 'SMAP', 'SPL3SMP.005')

# file suffix pattern for the SMAP files of interest
SMAP_FILE_SUFFIX = '*_R16010_001.h5'

# get paths to all HDF5 files ending with SMAP_FILE_SUFFIX
# found in all directories within SMAP_DATA_DIR
DATA_PATHS = sorted([str(filepath) for filepath in path(SMAP_DATA_DIR).rglob(SMAP_FILE_SUFFIX)])

# get the first in path DATA_PATHS to use for testing
TEST_DATA_PATH = DATA_PATHS[0]

# the HDF5 files contain many datasets, but
# the soil_moisture dataset is what we're interested in
SM_DATASET_NAME = 'soil_moisture'

# create variables to hold path to files containing EASE grid lat/lons (used for plotting)
# data source: https://github.com/TUW-GEO/ease_grid/tree/master/tests/test_data
EASE_GRID_DIR = os.path.join(DATA_DIR, 'ease_grid')
EASE_LATS_PATH = os.path.join(EASE_GRID_DIR, 'EASE2_M36KM.lats.964x406x1.double')
EASE_LONS_PATH = os.path.join(EASE_GRID_DIR, 'EASE2_M36KM.lons.964x406x1.double')

# store the EPSG projection code for the EASE Grid
# that this data uses (not used in this notebook)
EASE_EPSG = 'epsg:3410'

# create a path to an output directory in case
# we want to save files at some point
OUTPUT_DIR = os.path.join(DATA_DIR, 'output')

# check if OUTPUT_DIR exists and, if not, create it
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [6]:
# read EASE2 lat/lons and then reshape from a 1D to 2D array matching the EASE Grid shape (EASE_SHAPE)
EASE_SHAPE = (406, 964)
EASE_LATS = np.fromfile(EASE_LATS_PATH, dtype=np.float64).reshape(EASE_SHAPE)
EASE_LONS = np.fromfile(EASE_LONS_PATH, dtype=np.float64).reshape(EASE_SHAPE)

In [35]:
def print_dataset_names(src_path: str):
    '''
    Print name of datasets in file at src_path.
    '''
    
    # open file at src_path with rasterio
    with h5.File(src_path, 'r') as src:
        for subdataset in src.keys():
            print(subdataset)
            for dataset in src.get(subdataset).keys():
                print(" |--", dataset)

In [36]:
print_dataset_names(TEST_DATA_PATH)

Metadata
 |-- AcquisitionInformation
 |-- DataQuality
 |-- DatasetIdentification
 |-- Extent
 |-- GridSpatialRepresentation
 |-- Lineage
 |-- OrbitMeasuredLocation
 |-- ProcessStep
 |-- ProductSpecificationDocument
 |-- QADatasetIdentification
 |-- SeriesIdentification
Soil_Moisture_Retrieval_Data_AM
 |-- EASE_column_index
 |-- EASE_row_index
 |-- albedo
 |-- boresight_incidence
 |-- freeze_thaw_fraction
 |-- grid_surface_status
 |-- landcover_class
 |-- landcover_class_fraction
 |-- latitude
 |-- latitude_centroid
 |-- longitude
 |-- longitude_centroid
 |-- radar_water_body_fraction
 |-- retrieval_qual_flag
 |-- roughness_coefficient
 |-- soil_moisture
 |-- soil_moisture_error
 |-- static_water_body_fraction
 |-- surface_flag
 |-- surface_temperature
 |-- surface_water_fraction_mb_h
 |-- surface_water_fraction_mb_v
 |-- tb_3_corrected
 |-- tb_4_corrected
 |-- tb_h_corrected
 |-- tb_h_uncorrected
 |-- tb_qual_flag_3
 |-- tb_qual_flag_4
 |-- tb_qual_flag_h
 |-- tb_qual_flag_v
 |-- tb_ti

In [157]:
def read_metadata(src_path: str):
    with h5.File(src_path, 'r') as src:
        meta = src.get('Metadata')
        for item in meta.keys():
            print(item)
            for subitem in meta[item].keys():
                print('  |--', subitem)
                for subsubitem in meta[item][subitem].attrs.keys():
                    value = meta[item][subitem].attrs[subsubitem]
                    
                    if type(value) == bytes:
                        print('      |--', value.decode('UTF-8'))
                    else:
                        print('      |--', value)
    return meta

In [158]:
read_metadata(TEST_DATA_PATH)

AcquisitionInformation
  |-- platform
      |-- 14.6
      |-- The SMAP observatory houses an L-band radiometer that operates at 1.414 GHz and an L-band radar that operates at 1.225 GHz.  The instruments share a rotating reflector antenna with a 6 meter aperture that scans over a 1000 km swath.  The bus is a 3 axis stabilized spacecraft that provides momentum compensation for the rotating antenna.
      |-- SMAP
  |-- platformDocument
      |-- JPL CL#14-2285, JPL 400-1567
      |-- 2014-07-01
      |-- SMAP Handbook
  |-- radar
      |-- The SMAP 1.225 GHz L-Band Radar Instrument
      |-- SMAP SAR
      |-- L-Band Synthetic Aperture Radar
  |-- radarDocument
      |-- JPL CL#14-2285, JPL 400-1567
      |-- 2014-07-01
      |-- SMAP Handbook
  |-- radiometer
      |-- The SMAP 1.414 GHz L-Band Radiometer
      |-- SMAP RAD
      |-- L-Band Radiometer
  |-- radiometerDocument
      |-- JPL CL#14-2285, JPL 400-1567
      |-- 2014-07-01
      |-- SMAP Handbook
DataQuality
  |-- Completen

<Closed HDF5 group>

In [171]:
def read_dataset(src_path: str, target_dataset: str, mask_nodata: bool = True):
    '''
    Read dataset_name in HDF5 file at src_path.
    '''
    # open the file with rasterio
    with h5.File(src_path, 'r') as src:
        
        for subdataset in src.keys():

            for dataset in src.get(subdataset).keys():
                print(dataset)
                # compare the dataset_name with the target_dataset
                if dataset == target_dataset:
                
                    data = src[subdataset].get(dataset)
                    
                    print(data.ref)
                    print(np.array(data))
                    
                 

In [172]:
read_dataset(TEST_DATA_PATH, SM_DATASET_NAME)

AcquisitionInformation
DataQuality
DatasetIdentification
Extent
GridSpatialRepresentation
Lineage
OrbitMeasuredLocation
ProcessStep
ProductSpecificationDocument
QADatasetIdentification
SeriesIdentification
EASE_column_index
EASE_row_index
albedo
boresight_incidence
freeze_thaw_fraction
grid_surface_status
landcover_class
landcover_class_fraction
latitude
latitude_centroid
longitude
longitude_centroid
radar_water_body_fraction
retrieval_qual_flag
roughness_coefficient
soil_moisture
<HDF5 object reference>
[[-9999. -9999. -9999. ... -9999. -9999. -9999.]
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]
 ...
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]]
soil_moisture_error
static_water_body_fraction
surface_flag
surface_temperature
surface_water_fraction_mb_h
surface_water_fraction_mb_v
tb_3_corrected
tb_4_corrected
tb_h_corrected
tb_h_unco

In [ ]:
   # read the data into a numpy array
                    data = subdataset.read(1)
                    
                    # get the metadata associated with the subdataset
                    # (e.g., CRS, nodata value, filetype, shape)
                    meta = subdataset.meta
                    
                    # get additional metadata associated with the subdataset
                    # (e.g., units, valid min/max values, long name of dataset)
                    desc = subdataset.tags(bidx=1) # additional metadata (e.g., units)
                    
                    # the metadata in the file lacks projection information
                    # it's added here, but not really used in this script
                    meta['crs'] = EASE_EPSG
                    
                    # check whether to mask the data
                    if mask_nodata:
                        
                        # replace any cells containing the nodata value (-9999.0)
                        # with np.nan
                        data[data == meta['nodata']] = np.nan

    # return the data, metadata, and add'l metadata
    return data, meta, desc